In [1]:
import os
import pickle

In [2]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    metric_file_name: Path
    mlflow_uri: str
    transformed_data_test: Path
    transformed_data_train: Path
    model_path: Path
    all_params: dict
    target_column: str


In [7]:
from RegressionProject.constants import *
from RegressionProject.utils.common import read_yaml, create_directories, save_json, load_object_pkl, read_transformed_data

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config_model_eval = self.config.model_evaluation
        config_model_train = self.config.model_trainer
        config_model_transform = self.config.data_transformation

        params = self.params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config_model_eval.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config_model_eval.root_dir,
            metric_file_name = config_model_eval.metric_file_name,
            mlflow_uri=config_model_eval.mlflow_uri,
            transformed_data_test=config_model_transform.transformed_data_test,
            transformed_data_train=config_model_transform.transformed_data_train,
            model_path = config_model_train.trained_model_file_path,
            target_column = schema.name,

            all_params=params,
           
        )
        return model_evaluation_config


In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import dagshub
import mlflow.sklearn
import numpy as np

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        dagshub.init(repo_owner='iheb.aamrii', repo_name='Mlops-Regression-Project', mlflow=True)

        _, _, test_x, test_y = read_transformed_data(self.config.transformed_data_train, self.config.transformed_data_test)

        # Load the model from the pickle file
        model=load_object_pkl(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_math_scores = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_math_scores)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="best_regression_model")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-28 00:37:52,759: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-28 00:37:52,762: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 00:37:52,764: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-28 00:37:52,765: INFO: common: created directory at: artifacts]
[2024-06-28 00:37:52,766: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-28 00:37:53,329: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]
[2024-06-28 00:37:54,036: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/iheb.aamrii/Mlops-Regression-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "iheb.aamrii/Mlops-Regression-Project"

[2024-06-28 00:37:54,040: INFO: helpers: Initialized MLflow to track repo "iheb.aamrii/Mlops-Regression-Project"]


Repository iheb.aamrii/Mlops-Regression-Project initialized!

[2024-06-28 00:37:54,041: INFO: helpers: Repository iheb.aamrii/Mlops-Regression-Project initialized!]
[2024-06-28 00:37:54,684: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


C:\Users\iheba\IdeaProjects\Microservices\kafka\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'best_regression_model' already exists. Creating a new version of this model...
2024/06/28 00:38:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best_regression_model, version 2
Created version '2' of model 'best_regression_model'.


In [12]:
#Predict from logged model
_, _, test_x, test_y = read_transformed_data('artifacts/data_transformation/transformed_train_data.csv', 'artifacts/data_transformation/transformed_test_data.csv')


import mlflow
logged_model = 'runs:/3803771e0c3248debaa97c8ae4aa7412/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

array([76.34375, 58.71875, 76.75   , 76.53125, 87.125  , 79.0625 ,
       64.65625, 52.9375 , 74.25   , 48.9375 , 53.4375 , 36.09375,
       68.0625 , 51.84375, 86.5625 , 70.625  , 51.71875, 46.625  ,
       48.53125, 52.65625, 73.75   , 38.9375 , 57.21875, 36.21875,
       78.28125, 81.75   , 75.375  , 45.90625, 31.5625 , 49.53125,
       65.40625, 70.25   , 60.1875 , 83.09375, 83.6875 , 45.625  ,
       76.0625 , 69.75   , 65.6875 ,  3.5    , 78.6875 , 67.125  ,
       68.     , 60.34375, 79.5625 , 67.625  , 72.28125, 24.625  ,
       85.9375 , 70.6875 , 75.     , 66.65625, 81.     , 45.375  ,
       65.96875, 66.875  , 86.     , 58.625  , 81.9375 , 81.75   ,
       47.6875 , 72.25   , 70.625  , 55.53125, 86.5    , 64.0625 ,
       59.09375, 55.9375 , 57.125  , 82.     , 71.625  , 80.375  ,
       58.     , 46.8125 , 61.25   , 84.8125 , 84.1875 , 60.21875,
       59.5625 , 52.9375 , 63.3125 , 40.96875, 78.65625, 66.46875,
       75.     , 54.875  , 66.875  , 63.8125 , 75.5625 , 25.59